In [ ]:
# Imports
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output
import numpy as np
import dash_bootstrap_components as dbc
import dash_daq as daq

# Helper function to normalize to an array to a range (x,y)
def normalize_range(array, x, y):
    m = min(array)
    range = max(array) - m
    array = (array - m) / range
    range2 = y - x
    normalized = (array*range2) + x
    return list(normalized)

# Datasets and variables
snapshot = pd.read_csv("kamppi/simulation_output/clean_data.csv")
vehicle_types = sorted(snapshot["vehicle_type"].unique())
timeline = snapshot["timestep"].unique()
snapshot["vehicle_type"] = pd.Categorical(snapshot["vehicle_type"], categories=["electric","fuel"], ordered=True)
snapshot = snapshot.infer_objects()
snapshot["timestep"] = pd.to_datetime(snapshot["timestep"], format="%M").dt.minute

# Center point for heatmap
center_point= dict(lon=snapshot["lon"].mean(), lat = snapshot["lat"].mean())

# Style
external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.BOOTSTRAP]

# Hover layout
hovers=dict(bgcolor="white", font_size=16)

# Emission types
emission_types = ["Noise","Carbon monoxide", "Carbon dioxide", "Hydrocarbon", "Nitrogen oxides", "Particle matters"]
emission_cols = {"Noise": "vehicle_noise", 
"Carbon monoxide": "vehicle_CO",
"Carbon dioxide": "vehicle_CO2",
"Hydrocarbon": "vehicle_HC",
"Nitrogen oxides": "vehicle_NOx",
"Particle matters": "vehicle_PMx"}
emission_units = {"Noise": "dB",
"Carbon monoxide": "mg",
"Carbon dioxide": "mg",
"Hydrocarbon": "mg",
"Nitrogen oxides": "mg",
"Particle matters": "mg"}
objectives = ["Summary", "Air quality", "Livability", "Traffic"]
emission_maxes = 0.7*snapshot.max(axis=0, numeric_only=True)
emission_maxes = emission_maxes.to_dict()

# Main component
anim_app = Dash(__name__, external_stylesheets=external_stylesheets)
anim_app.layout = html.Div([

    html.Div([
        # Title
        html.H2("Visualization parameters", style={"display": "inline-block"}),

        # Top right info button
        dbc.Button([
            html.I(className="bi bi-info-circle-fill me-2")],
            id="info",
            color="white",
            title="""SUMO can simulate varying emissions with the HBEFA v4.2.-based emissions model. Choose which one you wish to analyze in the visualisation.""",
            n_clicks=0,
            style = {"display": "inline-block", "size": "20px"}
            ),
        ]),

        # Toggles and texts for parameters
        dbc.Row([

            # Emission dropdown column
            dbc.Col(
                html.Div([
                    html.Label("Objective", style={"padding-bottom": "2vh"}),
                    dcc.Dropdown(objectives, "Traffic", id="crossfilter-objective", style={"font-size": 18, "width": "75%"}),
                ]),
            ),

            # Timeline type column
            dbc.Col(
                html.Div([
                    html.Label("Aggregation method", style={"padding-bottom": "2vh"}),
                    dcc.RadioItems(options=["Sum", "Average"],
                    value="Sum",
                    id="crossfilter-timeline-type",
                    inline=True, labelStyle={"automargin": True, "padding-top": "1vh", "font-size": 18, "padding-right": "1vw"})
                ])
            ),

            # Timestep interval column
            dbc.Col(
                html.Div([
                    html.Label("Timestep interval",
                    style={"padding-bottom": "2vh"}),
                    html.Div([
                        daq.NumericInput(value=1,
                        id="crossfilter-timestep-range",
                        min=timeline.min(),
                        max=timeline.max(),
                        style={"display": "inline-block", "font-size": 18, "padding-right": "1vw"}),
                        html.P("minute(s)",
                        style={"display": "inline-block", "font-size": 18})
                    ]),
                ])
            ),

        # Row end
        ], justify="center", style={"automargin": True, "padding-top": "2vh", "font-size": 24}),

        # Dividing line between the params and plots
        html.Hr(),

        # Plots, centered
        html.Center([

        # Emission type dropdown
        html.Div([
            html.Label("Emission type",
            style={"font-size": 24, "padding-bottom": "2vh", "padding-top": "1vh"}),
            dcc.Dropdown(options=emission_types,
            value="Carbon dioxide",
            id="crossfilter-emission",
            style={"font-size": 18, "width": "50%", "margin": "auto", "text-align": "center"}),
            ], style={"padding-top": "2vh", "padding-bottom": "2vh"}),

        # Network plots
        html.Div([
                # The heatmap
                dcc.Loading(dcc.Graph(id="x-heatmap"), type="cube"),
        ], style={"padding-top": "3vh"}),
        
        # Drill-down plots   
        html.Div([
            # Title
            html.Label("Location: Network",
            style={"font-size": 24, "padding-bottom": "4vh", "display": "block", "padding-top": "2vh"}),
            # Reset button
            html.Button("Reset location",
            id="reset_button",
            n_clicks=0,
            style={"display": "block", "font-size": 18, "automargin": True, "padding": "1vh"}),
            # Histogram
            dcc.Graph(id="y-road"),
            # Stream graph
            dcc.Graph(id="y-flow"), 
        ], style={"padding-top": "3vh"}),
                
        ])

# Main component
], style = {"width": "95vw", "automargin": True, "padding-top": "2vh", "padding-left": "2vw"})

# Callbacks
@anim_app.callback(
    Output("x-heatmap", "figure"), 
    Input("crossfilter-emission", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"))
def get_heatmap(emission_name, timestep_range, timeline_type):
    zoom_level = 13
    if emission_name is None:
        emission_name = "Carbon dioxide"
    emission = emission_cols[emission_name]
    func = "sum"
    df = snapshot
    if timestep_range != 1:
        bins = np.arange(timeline.min(), timeline.max(), timestep_range, dtype=int)
        df['timestep'] = pd.cut(df['timestep'], bins=bins, labels=bins[:-1], right=False, include_lowest=True)
        if timeline_type == "Average":
            func = "mean"
        df = df.groupby(["vehicle_lane", "timestep"], observed=False).agg(
            {emission: func, "amount": func, "lon": "first", "lat": "first"}
            ).reset_index()
    fig = px.density_map(df, lat = "lat", lon = "lon", z = emission,
                            radius = normalize_range(df["amount"], 10, 18),
                            center = center_point,
                            zoom = zoom_level,
                            map_style = "open-street-map",
                            range_color = (0,emission_maxes[emission]),
                            hover_data = {"vehicle_lane": False, "amount": True, "lon": False, "lat": False, emission: True},
                            animation_frame="timestep",
                            labels={emission: emission_name,
                            "vehicle_type": "Vehicle type",
                            "amount": "Vehicle amount",
                            "timestep": "Timestep"},
                            height=750,
                            title="Network heatmap"
                            )
    sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time: "},
    font={"size": 18}
    )]
    fig.update_traces(unselected_marker_opacity=0.3, selector=dict(type="scatter"))
    fig.update_layout(hovermode="closest", sliders=sliders, title_x=0.12, hoverlabel=hovers)
    fig.layout["coloraxis"]["colorbar"]["title"] = f"{emission_name} ({emission_units[emission_name]})"
    fig["layout"]["uirevision"] = "Default"
    return fig

@anim_app.callback(
    Output("y-road", "figure"),
    Output("y-flow", "figure"),
    Input("x-heatmap", "clickData"),
    Input("crossfilter-emission", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"))
def get_barplot(clickData, emission_name, timestep_range, timeline_type):
    road = 0
    road_hist = 0
    road_name = "Network"
    if emission_name is None:
        emission_name = "Carbon dioxide"
    emission = emission_cols[emission_name]
    df = snapshot
    func = "mean"
    if timestep_range != 1:
        bins = np.arange(timeline.min(), timeline.max(), timestep_range, dtype=int)
        df['timestep'] = pd.cut(df['timestep'], bins=bins, labels=bins[:-1], right=False, include_lowest=True)
        if timeline_type == "Average":
            func = "mean"
    df = df.groupby(["vehicle_type", "vehicle_lane", "timestep"], observed=False).agg(
        {emission: func, "amount": func}
        ).reset_index()
    if clickData is None:
        road = df
    else:
        road_id = clickData["points"][0]["customdata"][0]
        road_name = "Road"
        road = df[df["vehicle_lane"] == road_id]
    road_hist = road.groupby(["vehicle_type"], observed=False).agg(
        {emission: "sum", "amount": "sum"}
        ).reset_index()
    road_stream = road.groupby(["vehicle_type", "timestep"], observed=False).agg(
        {emission: "sum", "amount": "sum"}
        ).reset_index()
    road_stream.sort_values(by=["vehicle_type"])
    road_hist.sort_values(by=["vehicle_type"])
    road_hist_non_zero = road_hist[road_hist[emission] != 0]
    road_hist["average_vehicle"] = np.round(road_hist_non_zero[emission]/road_hist_non_zero["amount"], 4)
    average_dict = {
        road_hist["vehicle_type"][0]: road_hist["average_vehicle"][0], 
        road_hist["vehicle_type"][1]: road_hist["average_vehicle"][1]
    }
    road_stream["average_vehicle"] = road_stream["vehicle_type"].map(average_dict)
    hist = px.bar(road_hist, 
    x=emission, 
    y="vehicle_type", 
    color="average_vehicle", 
    hover_data="amount", 
    labels={"average_vehicle": "Average vehicle emission",
    emission: emission_name,
    "vehicle_type": "Mobility mode",
    "amount": "Vehicle amount"}, 
    title="Mobility mode average")
    hist.update_layout(hoverlabel=hovers, 
    title_x=0.11, bargap=0.5, 
    xaxis_title=f"{emission_name} ({emission_units[emission_name]})", 
    yaxis_title="Mobility mode", 
    xaxis=dict(range=[0, snapshot[emission].max()]))
    stream = px.area(road_stream, 
    x="timestep", 
    y=emission, 
    color="vehicle_type", 
    color_discrete_sequence=px.colors.sequential.Plasma_r, 
    hover_data=["average_vehicle", "amount"],
    labels={"average_vehicle": "Average vehicle emission",
    emission: emission_name,
    "vehicle_type": "Mobility mode",
    "amount": "Amount"}, 
    title="Mobility mode time series")
    stream.update_layout(hoverlabel=hovers,
    title_x=0.11,
    yaxis_title=f"{emission_name} ({emission_units[emission_name]})")
    return hist, stream

@anim_app.callback(Output("x-heatmap","clickData"),
             [Input("reset_button","n_clicks")])
def update(reset):
    return None

anim_app.run(debug=True)